In [6]:
# Chase Abram
# using Pkg
# Pkg.add("DataFrames")
using CSV
using DataFrames
df = DataFrame()
df = CSV.read("psetOne.csv", DataFrame)

# Pkg.add("ForwardDiff")
using ForwardDiff
using LinearAlgebra
using Optim

using Random

In [7]:
names(df)
df17 = df[df."Market" .== 17, :]
# df17."shares"
X = convert(Matrix, df17[:, filter(x -> x in ["Constant", "EngineSize", "SportsBike", "Brand2", "Brand3"], names(df17))])
Br = convert(Matrix, df17[:, filter(x -> x in ["Brand2", "Brand3"], names(df17))])
Br = Bool.(Br)

7×2 BitArray{2}:
 0  0
 0  0
 1  0
 1  0
 0  1
 0  1
 0  1

In [148]:
function u_bar_jt(delta, sigma, X, zeta)
    return delta .+ X*sigma*zeta
end

function Prij(u_bar)
    return exp.(u_bar)./sum(exp.(u_bar))
end

# Why is this jumping to 0?
# Shares
function s_logit(p, alpha, beta, X, xi)
    
    println("p in s: ", p)
    println("X*beta - alpha .* p + xi: ", X*beta - alpha .* p + xi)
    # Compute numerators for each s_j
#     nums = exp.(X*beta .- alpha.*p[2:end] .+ xi)
    nums = exp.(X*beta - alpha.*p + xi)
    println("nums: ", nums)
    
    # return s_j, starting with s_0
#     return [1/(1 + sum(nums)); nums./(1 + sum(nums))]
    return nums./(1 + sum(nums))
end

# Jacobian of shares
function s_logit_jac(p, alpha, beta, X, xi)
    # Get shares
    s = s_logit(p, alpha, beta, X, xi)
    
    println("s: ", s)
    
    # Jacobian is the outer product times alpha in this case...
    jac = alpha .* s * s'
    
    # ... except for an extra term on the diag
    for j in 1: length(p)
        jac[j,j] -= alpha*s[j]
    end
    
    return jac
end

# Hessian of shares (n by n by n tensor)
function s_logit_hess(p, alpha, beta, X, xi)
    
    # Initialize
    hess = zeros(length(p), length(p), length(p))
    
    # Shares
    s = s_logit(p, alpha, beta, X, xi)

    # Jacobian of shares
    sj = s_logit_jac(p, alpha, beta, X, xi)
    
    # Fill in
    for j in 1:length(p)
        for k in 1:length(p)
            for l in 1:length(p)
                
                if j == k
                    # on diag of jacobian
                    hess[j,k,l] = alpha*sj[l,j]*(2*s[j] - 1)
                else
                    # off diag of jacobian
                    hess[j,k,l] = alpha*(sj[l,j]*s[k] + s[j]*sj[l,k])
                end
            end
        end
    end
    
    return hess
end

# Ownership structure
function Omega_star(br)
    
    # Initialize
#     Om = zeros(size(br,1) + 1,size(br,1) + 1)
    Om = zeros(size(br,1),size(br,1))
    
    for i in 1:size(Om,1)
        for j in 1:size(Om,2)
            if br[i,1] && br[j,1]
                # Brand2
                Om[i,j] = 1
            elseif br[i,2] && br[j,2]
                # Brand3
                Om[i,j] = 1
            elseif (!br[i,1]&& !br[j,1]) && (!br[i,2]&& !br[j,2])
                # Brand1
                Om[i,j] = 1
            end
        end
    end
    
    # Outside option
#     Om[1,1] = 1
    
    return Om
end

# Negative of Omega in Nevo (2001)
function Omega(p, alpha, beta, X, xi, br)
    return Omega_star(br) .* s_logit_jac(p, alpha, beta, X, xi)
end

# Fixed point expression (want zero at FP)
function fp_logit(p, alpha, beta, X, xi, br)
    return s_logit(p, alpha, beta, X, xi) + Omega(p, alpha, beta, X, xi, br)*p
end

# (Jacobian of Omega)*p (this is used in the gradient of the FP expression)
function jac_om_p(p, alpha, beta, X, xi, br)
    jacomp = zeros(length(p), length(p))
    sj = s_logit_jac(p, alpha, beta, X, xi)
    sh = s_logit_hess(p, alpha, beta, X, xi)
    oms = Omega_star(br)
    for i in 1:length(p)
        for k in 1:length(p)
#             jacomp[i,k] = sum([sh[i,j,k]*p[j]*oms[i,j] for j in 1:length(p)]) + sj[i,k]*oms[i,k]
            jacomp[i,k] = (sh[i,:,k]' * (oms[i,:] .* p))[1] #+ oms[i,k] * sj[i,k]
        end
    end
    return jacomp
end

function J_fp_logit(p, alpha, beta, X, xi, br)
    sj = s_logit_jac(p, alpha, beta, X, xi)
    jacomp = jac_om_p(p, alpha, beta, X, xi, br)
    om = Omega(p, alpha, beta, X, xi, br)
    println("sj: ", sj)
    println("jacomp: ", jacomp)
    println("om: ", om)
    return sj + jacomp + om
end

function solve_fp_logit(p0, alpha, beta, X, xi, br, tol = 1e-14, maxiter = 100)
    p = p0
    pnew = zeros(length(p0))
    J = zeros(length(p0), length(p0))
    
    it = 0
    diff = Inf
    println("p0: ", p0)
    while diff > tol && it < maxiter
        println("p: ", p)
        J .= J_fp_logit(p, alpha, beta, X, xi, br)
        println("J: ", J)
#         println("J inv: ", inv(J))
#         println("J under fp: ", J \ fp_logit(p, alpha, beta, X, xi, br))
#         println("size(p): ", size(p))
#         println("size(J under fp): ", size(J \ fp_logit(x, alpha, beta, X, xi, br)))
        pnew .= (p .- (J \ fp_logit(p, alpha, beta, X, xi, br)))[:,1]
        pnew .= min.(pnew, 50)
        
#         pnew -= J*p
#         xnew = min.(max.(xnew, -1000), 1000)
        println("pnew: ", pnew)
        diff = maximum(abs.(pnew - p))
        println("diff: ", diff)
        it += 1
        p .= pnew
    end
    return p
end



# Squared distance of FP expression from origin
function fp_logit_2(p, alpha, beta, X, xi, br)
    return (fp_logit(p, alpha, beta, X, xi, br)' * fp_logit(p, alpha, beta, X, xi, br))[1]
end


# function fp_logit_2_jac(storage, p, alpha, beta, X, xi, br)
#     sp = s_logit(p, alpha, beta, X, xi)
#     Om = Omega_star(br).* s_logit_jac(p, alpha, beta, X, xi)
#     s_jac = s_logit_jac(p, alpha, beta, X, xi)
#     jacomp = jac_om_p(p, alpha, beta, X, xi, br)
    
#     jac = 2*(sp + Om*p)' * (s_jac + jacomp + Om)
#     for i in 1:length(p)
#         storage[i] = jac[i]
#     end
# #     storage = (sp + Om*p)' * (s_jac + jacomp + Om)
# #     storage = 0
# #     return storage
    
# end

# function fp_logit_2_jac(storage, p, alpha, beta, X, xi, br)
#     s = s_logit(p, alpha, beta, X, xi)
#     oms = Omega_star(br)
#     sj = s_logit_jac(p, alpha, beta, X, xi)
#     sh = s_logit_hess(p, alpha, beta, X, xi)
    
#     for k in 1:length(p)
#         storage[k] = (2 .* (s + (oms .* sj)*p)' * (sj[k,:] + (oms .* sh[:,:,k])*p + sj[:,k]))[1]
#     end
#     println("stor: ", storage)
# #     return storage
# end

function fp_logit_2_jac(p, alpha, beta, X, xi, br)
    s = s_logit(p, alpha, beta, X, xi)
    oms = Omega_star(br)
    sj = s_logit_jac(p, alpha, beta, X, xi)
    sh = s_logit_hess(p, alpha, beta, X, xi)
    
    out = zeros(length(p))
    for k in 1:length(p)
        out[k] = (2 .* (s + (oms .* sj)*p)' * (sj[k,:] + (oms .* sh[:,:,k])*p + sj[:,k]))[1]
    end
#     println("stor: ", storage)
    return out
end

function basic_newton_zero(f, g, init, tol = 1e-14, maxiter = 10000)
    x = init
    x_new = init
    diff = Inf
    it = 1
#     println("x init: ", init)
    while diff > tol && it < maxiter
#         println("it: ", it)
#         println("g(x): ", g(x))
        x_new -= 0.5.*g(x)
#         println("x_new: ", x_new)
        diff = maximum(abs.(x_new - x))
#         println("diff: ", diff)
        it += 1
        x = x_new
    end
    return x
end


# function solve_fp_logit(alpha, beta, X, xi, br, tol = 1e-14, maxit = 100)
    
#     f(x) = fp_logit_2(x, alpha, beta, X, xi, br)[1,1]
    
# #     g!(storage, x) = fp_logit_2_jac(storage, x, alpha, beta, X, xi, br)
#     g!(x) = fp_logit_2_jac(x, alpha, beta, X, xi, br)
    
#     out = basic_newton_zero(f, g!, p)
# #     Optim.Options(f_tol = tol)
# #     Optim.Options(time_limit = 10.0)
# #     opt = optimize(f, g!, p, LBFGS(), Optim.Options(time_limit = 5.0))
# #     opt = optimize(f, x -> ForwardDiff.gradient(f, p), p, BFGS(), Optim.Options(time_limit = 10.0))
# #     return opt, opt.minimizer, opt.minimum
#     return out
# end



basic_newton_zero (generic function with 3 methods)

In [149]:
p = zeros(size(X,1),1) .* 1.8
# p[7] = 1
alpha = 3
beta = [1 1 2 -1 1]'


# seed maintains results across runs
rng = MersenneTwister(1234)
xi = randn(rng, size(df17,1))

println("delta: ", X*beta - alpha .* p + xi)

s_logit(p, alpha, beta, X, xi)
# println("sum shares: ", sum(s_logit(p, alpha, beta, X, xi)))

s_logit_jac(p, alpha, beta, X, xi)

s_logit_hess(p, alpha, beta, X, xi)

Omega_star(Br)

Omega(p, alpha, beta, X, xi, Br)

fp_logit(p, alpha, beta, X, xi, Br)' * fp_logit(p, alpha, beta, X, xi, Br)

out = solve_fp_logit(p, alpha, beta, X, xi, Br)
println("out: ", out)
println("fp_logit: ", fp_logit(out, alpha, beta, X, xi, Br))

# inv(J_fp_logit(p, alpha, beta, X, xi, Br))

# fp_logit_2(p, alpha, beta, X, xi, Br)
# st = zeros(length(p))
# fp_logit_2_jac(st, p, alpha, beta, X, xi, Br)
# println(st)

# jac_om_p(p, alpha, beta, X, xi, Br)

# fp_logit_jac(p, alpha, beta, X, xi, Br)


# op = solve_fp_logit(alpha, beta, X, xi, Br)
# println("op: ", op)
# println("opam: ", opam)
# println("test on fp: ", fp_logit(opam, alpha, beta, X, xi, Br))

# op = optimize(x -> (fp_logit(x, alpha, beta, X, xi, Br)' * fp_logit(x, alpha, beta, X, xi, Br))[1,1], p, LBFGS())

# pmin = op.minimizer
# println("p min: ", pmin)
# println("min: ", op.minimum)
# println("fp_logit: ", fp_logit(pmin, alpha, beta, X, xi, Br))
# df17[3, "Brand2"]

# Gradient must be wrong.
# Check whole construction and parameter choice.


delta: [6.867347201951246; 7.098256184143183; 4.505521246495766; 6.0970857061347585; 9.364401313253515; 16.21187749957435; 12.532813282169538]
p in s: [0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0]
X*beta - alpha .* p + xi: [6.867347201951246; 7.098256184143183; 4.505521246495766; 6.0970857061347585; 9.364401313253515; 16.21187749957435; 12.532813282169538]
nums: [960.397443038427; 1209.8554687510211; 90.51551264834796; 444.5603010397264; 11665.619763891997; 1.0983201197265705e7; 277288.36766584666]
p in s: [0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0]
X*beta - alpha .* p + xi: [6.867347201951246; 7.098256184143183; 4.505521246495766; 6.0970857061347585; 9.364401313253515; 16.21187749957435; 12.532813282169538]
nums: [960.397443038427; 1209.8554687510211; 90.51551264834796; 444.5603010397264; 11665.619763891997; 1.0983201197265705e7; 277288.36766584666]
s: [8.518042034442971e-5; 0.00010730557242861756; 8.02808198935337e-6; 3.9429335829140645e-5; 0.0010346574767242985; 0.9741318050063816; 0.0245935054134172

jacomp: [3.100924721782092e-60 -5.383719513558273e-122 -2.013918787455252e-123 -9.891214403203234e-123 -2.59553419324754e-121 -2.44369993500535e-118 -6.1695088150754144e-120; -5.383719513558273e-122 3.906373095875928e-60 -2.5370232670701727e-123 -1.2460403685004e-122 -3.269710119277355e-121 -3.078437736151361e-118 -7.772005260459709e-120; -2.013918787455252e-123 -2.5370232670701727e-123 2.9225587064043158e-61 -1.8644538235923534e-123 -2.446238375592129e-122 -2.3031376642981334e-119 -5.8146370258787235e-121; -9.891214403203234e-123 -1.2460403685004e-122 -1.8644538235923534e-123 1.4353932716184904e-60 -1.2014520349601224e-121 -1.1311691702549214e-118 -2.855816324770661e-120; -2.59553419324754e-121 -3.269710119277354e-121 -2.446238375592129e-122 -1.2014520349601224e-121 3.7665873626566212e-59 -5.936557716903372e-117 -1.4987783336647323e-118; -2.4436999350053502e-118 -3.078437736151361e-118 -2.3031376642981334e-119 -1.1311691702549214e-118 -5.936557716903372e-117 3.546248520732981e-56 -1.4

In [430]:
# NM: opam: [0.3720742469890362; 0.3724406061806239; 0.3447723065789802; 
# 0.3424326539927414; 2.4893064284464703; 2.5105275013368717; 4.366510826679992]

# BFGS: opam: [0.37346117252339095; 0.37346116923202516; 5.8690207005673365; 
# 0.34160603728869476; 2.5287477985474984; 2.5287470025674392; 2.528747016448347]







sHat (generic function with 1 method)

In [707]:
function f1(x, y, z) 
    return (x[1] - 5 - y)^2 + (x[2] - 3 - z)^2
end

function g1(storage, x, y, z)
    storage[1] = 2*(x[1] - 5 - y)
    storage[2] = 2*(x[2] - 3 - z)
end

function g2(x,y,z)
    out = zeros(2)
    out[1] = 2*(x[1] - 5 - y)
    out[2] = 2*(x[2] - 3 - z)
    
    return out
end

function tester()
    a = 1000
    b = -1000
    f(x) = f1(x,a,b)
#     g!(storage, x) = g1(storage, x, a, b)
    g!(x) = g2(x,a,b)
    
    out = basic_newton_zero(f, g!, [-200.0, 1000.0])
    
#     opt = optimize(f, g!, [100.0, -20.0], LBFGS())
#     return opt, opt.minimizer, opt.minimum
    return out
end

tester()



x init: [-200.0, 1000.0]
it: 1
g(x): [-2410.0, 3994.0]
x_new: [1005.0, -997.0]
diff: 1997.0
it: 2
g(x): [0.0, 0.0]
x_new: [1005.0, -997.0]
diff: 0.0


2-element Array{Float64,1}:
 1005.0
 -997.0

In [38]:
p = ones(size(X,1),1)
# p[1] = 50
# p[2] = 30
# p[3] = 90
# p[4] = -12
# p[5] = -40
# p[6] = 0
# p[7] = 1
s(x) = s_logit(x, alpha, beta, X, xi)
# println(s_logit_jac(p, alpha, beta, X, xi))
ForwardDiff.jacobian(s, p)

# ForwardDiff.gradient(fp, p)


# fp(x) = fp_logit_2(x, alpha, beta, X, xi, Br)
# println(fp_logit_2_jac(p, alpha, beta, X, xi, Br))
# ForwardDiff.gradient(fp, p)

fp(x) = fp_logit(x, alpha, beta, X, xi, Br)
J_ex = J_fp_logit(p, alpha, beta, X, xi, Br)
println(J_ex)
println(maximum(abs.(ForwardDiff.jacobian(fp, p) .- J_ex)))
println(inv(J_ex))

[0.00025530622161152114 -1.0965196505156214e-7 -4.100629194543775e-9 -2.0139939506954153e-8 -5.284882069011349e-7 -0.0004975725614462154 -1.2562010007891298e-5; -1.0965196505156214e-7 0.00032159214614629504 -5.1657453820817555e-9 -2.537117953554182e-8 -6.657601516169134e-7 -0.0006268143349711841 -1.5824923959053317e-5; -4.102414347598334e-9 -5.167994218957853e-9 2.4080002939195287e-5 -3.798220030691272e-9 -4.983062610163822e-8 -4.6915620716015493e-5 -1.184459398741835e-6; -2.0148707155240248e-8 -2.53822245329061e-8 -3.798220030691274e-9 0.00011825231170332975 -2.447394650108813e-7 -0.00023042262987567737 -5.817385456564767e-6; 1.0572036578993093e-6 1.3318065727522822e-6 9.963930220984128e-8 4.893711242390426e-7 -0.006199265020645103 0.006042950692029759 0.00015256389309296163; 0.0009953590736030823 0.0012538981931894371 9.381057547522316e-5 0.00046074376042034546 0.006042950692029759 -0.1532131616148883 0.1436391693876953; 2.512942154146277e-5 3.165665246077917e-5 2.3683970525637177e-6

In [183]:
nI = 20
# nJ = 10
nN = 5

delta_1 = zeros(3)
X_1 = zeros(3,nN)
sigma_1 = 0.0 .* I(size(X,2)).* 0.1
zeta_1 = zeros(nN, nI)

delta_2 = zeros(3)
delta_2[1] = 40
delta_2[2] = 20
delta_2[3] = 20
X_2 = zeros(nJ,nN)
sigma_2 = 0.0 .* I(size(X,2)).* 0.1
zeta_2 = zeros(nN, nI)

sigma_3 = 0.1 .* I(size(X,2))
delta_3 = zeros(size(X,2))
X_3a = zeros(size(X))

rng = MersenneTwister(1234)
zeta_3 = randn(rng, nN, nI)

rng = MersenneTwister(1234)
X_3b = randn(rng, size(X_3a))
X_3c = X_3b .* 10
X_3d = abs.(X_3c)
    




function sHat(delta, X, sigma, zeta)
    
    u_bar = zeros(size(zeta,2), size(delta,1))
    
    for i in 1:size(u_bar, 1)
        for j in 1:size(u_bar, 2)
#             println("delta[j]: ", size(delta[j]))
#             println("X[j,:]: ", size(X[j,:]))
#             println("sigma: ", size(sigma))
#             println("zeta[:,i]: ", size(zeta[:,i]))
            
            u_bar[i,j] = delta[j] + X[j,:]'*sigma*zeta[:,i]
        end
    end
    
    pr = zeros(size(zeta,2), size(delta,1))
    
    for i in 1:size(pr,1)
        for j in 1:size(pr,2)
            pr[i,j] = exp(u_bar[i,j])/(1 + sum(exp.(u_bar[i,:])))
        end
    end
    
#     println("pr: ", pr)
    
    s = zeros(size(delta))
    
    for j in 1:length(s)
        s[j] = 1/size(zeta,2) * sum(pr[:,j])
    end
    
    return s
end

println("s1: ", sHat(delta_1, X_1, sigma_1, zeta_1))
println("s2: ", sHat(delta_2, X_2, sigma_2, zeta_2))
println("s3a: ", sHat(delta_3, X_3a, sigma_3, zeta_3))
println("s3b: ", sHat(delta_3, X_3b, sigma_3, zeta_3))
println("s3c: ", sHat(delta_3, X_3c, sigma_3, zeta_3))
println("s3d: ", sHat(delta_3, X_3d, sigma_3, zeta_3))

s1: [0.25, 0.25, 0.25]
s2: [0.999999995877693, 2.0611536139418496e-9, 2.0611536139418496e-9]
s3a: [0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666]
s3b: [0.17049141609366947, 0.1672576505059677, 0.17607521710600005, 0.16262522593808482, 0.16102535927576098]
s3c: [0.157519607527216, 0.22113939342614863, 0.24138842201313673, 0.20933630482105567, 0.0927739387866564]
s3d: [0.11106304301591606, 0.11056581613744365, 0.10059910933267713, 0.22341569460653107, 0.19352177497916223]
